<a href="https://colab.research.google.com/github/BagaskaraAdhi/DataCleansing-Penjualan-Produk-Cetakan/blob/main/2318090DataCleansing_Penjualan_Produk_Cetakan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Datasets Dengan API Kaggle

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bagaskaraadhipradana","key":"d044974076f03086a29100ab7518ca2f"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d jabirmuktabir/data-penjualan-produk-cetakan
!unzip data-penjualan-produk-cetakan.zip

Dataset URL: https://www.kaggle.com/datasets/jabirmuktabir/data-penjualan-produk-cetakan
License(s): apache-2.0
  0% 0.00/6.74k [00:00<?, ?B/s]
100% 6.74k/6.74k [00:00<00:00, 25.1MB/s]
Archive:  data-penjualan-produk-cetakan.zip
  inflating: data_penjualan.csv      


## Import Library

In [16]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

## Baca File CSV

In [17]:
fn = "data_penjualan.csv"
df = pd.read_csv(fn, sep=';')
print("Shape:", df.shape)
print(df.columns.tolist())
df.head(8)

Shape: (1076, 5)
['Tanggal', 'Jenis Produk', 'Jumlah Order', 'Harga', 'Total']


,Tanggal,Jenis Produk,Jumlah Order,Harga,Total
0,05/08/2022,Foodpak260,1000,1800,1800000
1,05/08/2022,FoodpakMatte245,1000,1900,1900000
2,05/08/2022,CraftLaminasi290,5000,750,3750000
3,05/08/2022,CraftLaminasi290,1000,1200,1200000
4,07/08/2022,Dupleks310,1000,1550,1550000
5,08/08/2022,Dupleks310,500,1900,950000
6,08/08/2022,Ivory270,1000,1700,1700000
7,09/08/2022,Kinstruk130,1000,1000,1000000


## Rapikan nama kolom & tampilkan dengan jelas

In [18]:
df.columns = df.columns.str.strip().str.replace('\n', ' ').str.replace('\r','')
for c in df.columns:
    print(repr(c))

'Tanggal'
'Jenis Produk'
'Jumlah Order'
'Harga'
'Total'


## Ubah kolom tanggal ke datetime

In [20]:
df['Tanggal'] = pd.to_datetime(df['Tanggal'], dayfirst=True, errors='coerce')
print("Jumlah tanggal null setelah parsing:", df['Tanggal'].isnull().sum())
df.loc[df['Tanggal'].isnull()].head(8)

Jumlah tanggal null setelah parsing: 0


,Tanggal,Jenis Produk,Jumlah Order,Harga,Total


## Konversi kolom numerik

In [21]:
num_cols = ['Jumlah Order', 'Harga', 'Total']

for col in num_cols:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].str.replace(r'[^0-9\-]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df[num_cols].head()
print(df[num_cols].isnull().sum())

Jumlah Order    0
Harga           0
Total           0
dtype: int64


## Periksa missing values & strategi penanganan

In [22]:
print("Missing per kolom:")
print(df.isnull().sum())
df = df.dropna(subset=['Tanggal'])
for col in ['Jumlah Order','Harga']:
    med = df[col].median()
    df[col] = df[col].fillna(med)

Missing per kolom:
Tanggal         0
Jenis Produk    0
Jumlah Order    0
Harga           0
Total           0
dtype: int64


## Hilangkan duplikat

In [23]:
print("Duplicated rows before:", df.duplicated().sum())
df = df.drop_duplicates()
print("After:", df.duplicated().sum())

Duplicated rows before: 40
After: 0


## Normalisasi teks

In [24]:
df['Jenis Produk'] = df['Jenis Produk'].astype(str).str.strip()
df['jenis_produk_clean'] = df['Jenis Produk'].str.lower().str.replace(r'\s+', ' ', regex=True)
df[['Jenis Produk','jenis_produk_clean']].head()

,Jenis Produk,jenis_produk_clean
0,Foodpak260,foodpak260
1,FoodpakMatte245,foodpakmatte245
2,CraftLaminasi290,craftlaminasi290
3,CraftLaminasi290,craftlaminasi290
4,Dupleks310,dupleks310


## Verifikasi konsistensi Total = Jumlah Order * Harga

In [25]:
df['expected_total'] = df['Jumlah Order'] * df['Harga']
mismatch_mask = (df['Total'].notnull()) & (df['Total'] != df['expected_total'])
print("Jumlah mismatch Total vs expected:", mismatch_mask.sum())
df.loc[mismatch_mask, ['Tanggal','Jenis Produk','Jumlah Order','Harga','Total','expected_total']].head(10)
df.loc[mismatch_mask, 'Total'] = df.loc[mismatch_mask, 'expected_total']
df['Total'] = df['Total'].fillna(df['expected_total'])


Jumlah mismatch Total vs expected: 0


## Tangani nilai tidak wajar

In [27]:
for col in ['Harga','Jumlah Order','Total']:
    q_low = df[col].quantile(0.01)
    q_high = df[col].quantile(0.99)
    print(col, " -> 1%:", q_low, "99%:", q_high)

df = df[(df['Harga'] >= df['Harga'].quantile(0.01)) & (df['Harga'] <= df['Harga'].quantile(0.99))]

Harga  -> 1%: 250.0 99%: 3800.0
Jumlah Order  -> 1%: 500.0 99%: 17899.999999999636
Total  -> 1%: 300000.0 99%: 7934499.99999998


## Cek ringkasan & statistik akhir

In [28]:
print(df.info())
display(df.head())
display(df.describe(include='all'))
print("Missing akhir per kolom:\n", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 1012 entries, 0 to 1075
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Tanggal             1012 non-null   datetime64[ns]
 1   Jenis Produk        1012 non-null   object        
 2   Jumlah Order        1012 non-null   int64         
 3   Harga               1012 non-null   int64         
 4   Total               1012 non-null   int64         
 5   jenis_produk_clean  1012 non-null   object        
 6   expected_total      1012 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 63.2+ KB
None


,Tanggal,Jenis Produk,Jumlah Order,Harga,Total,jenis_produk_clean,expected_total
0,2022-08-05,Foodpak260,1000,1800,1800000,foodpak260,1800000
1,2022-08-05,FoodpakMatte245,1000,1900,1900000,foodpakmatte245,1900000
2,2022-08-05,CraftLaminasi290,5000,750,3750000,craftlaminasi290,3750000
3,2022-08-05,CraftLaminasi290,1000,1200,1200000,craftlaminasi290,1200000
4,2022-08-07,Dupleks310,1000,1550,1550000,dupleks310,1550000


,Tanggal,Jenis Produk,Jumlah Order,Harga,Total,jenis_produk_clean,expected_total
count,1012,1012,1012.000000,1012.000000,1.012000e+03,1012,1.012000e+03
unique,NaN,89,NaN,NaN,NaN,86,NaN
top,NaN,Dupleks310,NaN,NaN,NaN,dupleks310,NaN
freq,NaN,150,NaN,NaN,NaN,152,NaN
mean,2023-04-08 17:50:02.371541504,NaN,1974.081028,1441.778656,2.362422e+06,NaN,2.362422e+06
min,2022-08-05 00:00:00,NaN,100.000000,250.000000,1.000000e+05,NaN,1.000000e+05
25%,2022-12-12 00:00:00,NaN,1000.000000,950.000000,1.312500e+06,NaN,1.312500e+06
50%,2023-04-25 12:00:00,NaN,1000.000000,1500.000000,1.870000e+06,NaN,1.870000e+06
75%,2023-08-07 00:00:00,NaN,2000.000000,1800.000000,3.000000e+06,NaN,3.000000e+06
max,2023-11-15 00:00:00,NaN,40000.000000,3800.000000,1.100000e+07,NaN,1.100000e+07


Missing akhir per kolom:
 Tanggal               0
Jenis Produk          0
Jumlah Order          0
Harga                 0
Total                 0
jenis_produk_clean    0
expected_total        0
dtype: int64


## Simpan file hasil cleaning & download

In [29]:
clean_fn = "data_penjualan_bersih.csv"
df.drop(columns=['expected_total'], errors='ignore', inplace=True)
df.to_csv(clean_fn, index=False)
print("Saved:", clean_fn)

from google.colab import files
files.download(clean_fn)

Saved: data_penjualan_bersih.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>